**1) Install and Import Dependencies**

In [1]:
import tensorflow as tf
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import numpy as np

**2) Keypoints using MP Holistic**

In [2]:
mp_holistic = mp.solutions.holistic     # Holistic Model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Color conversion BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make Prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color conversion RGB 2 BGR
    return image, results

In [4]:
def draw_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #Draw pose connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw left hand connection
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw right hand connection

In [5]:
def draw_style_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                              mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2)) #Draw pose connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2)) #Draw left hand connection
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2)) #Draw right hand connection

In [6]:
#Video path
VIDEO_PATH = os.path.join(os.getcwd(), 'video_dataset.mp4')
VIDEO_PATH

'F:\\Nayaka\\CodePython\\ResearchHumanActivity\\video_dataset.mp4'

In [ ]:
cap = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make Detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # New export keypoints
        keypoints = extract_keypoint_result(results)
        print(keypoints)
                    
        # Draw Landmarks
        draw_style_landmark(image, results)
                    
        # Show to Screen
        cv2.imshow('OpenCV Video Capture', image)

        # Break Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

**3) Extract Keypoint Values**

In [ ]:
# Data accumulation from pose landmarks
len(results.pose_landmarks.landmark) * 4

In [ ]:
# Data accumulation from left hand landmarks
len(results.left_hand_landmarks.landmark) * 3

In [ ]:
# Data accumulation from right hand landmarks
len(results.right_hand_landmarks.landmark) * 3

In [7]:
def extract_keypoint_result(results):
    pose_value = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    rh_value = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    lh_value = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose_value, lh_value, rh_value])

In [ ]:
# Check shape from extract keypoint result
result_test = extract_keypoint_result(results).shape

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

**4) Setup folders for Collection**

In [8]:
# Path for exported data
DATA_PATH = os.path.join('Dataset')

# Action that try to detect
actions = np.array(['Walking', 'Trash'])

# Number of video in dataset
no_sequences = 30

#Videos are going to be 60 Frames in length (Equal to 2 Sec)
sequence_length = 60

In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

**5) Collecting Dataset Value**

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw Landmarks
                draw_style_landmark(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoint_result(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

**6) Preprocess Data and Create Labels and Features**

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:
label_map = {label:num for num, label in enumerate(actions)}

In [16]:
label_map

{'Walking': 0, 'Trash': 1}

In [17]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [18]:
np.array(sequences).shape

(60, 60, 258)

In [19]:
np.array(labels).shape

(60,)

In [20]:
X = np.array(sequences)

In [21]:
X.shape

(60, 60, 258)

In [22]:
y = to_categorical(labels).astype(int)

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [127]:
y_train.shape, X_train.shape

((48, 2), (48, 60, 258))

In [128]:
y_test.shape, X_test.shape

((12, 2), (12, 60, 258))

In [129]:
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [130]:
y_train.shape, y_test.shape

((48, 2), (12, 2))

**7) Build and Train RNN**

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import TensorBoard

In [132]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [133]:
model = Sequential()
model.add(SimpleRNN(64, return_sequences=True, activation='relu', input_shape=(60, 258)))
model.add(SimpleRNN(128, return_sequences=True, activation='relu'))
model.add(SimpleRNN(64, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='sigmoid'))

In [134]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 60, 64)            20672     
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 60, 128)           24704     
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 64)                12352     
                                                                 
 dense_12 (Dense)            (None, 128)               8320      
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 2)                

In [135]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [136]:
X_train.shape, y_train.shape

((48, 60, 258), (48, 2))

In [137]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 3s 77ms/step - loss: 0.7123 - acc: 0.3958
Epoch 2/2000
2/2 [==============================] - 0s 83ms/step - loss: 0.6976 - acc: 0.5833
Epoch 3/2000
2/2 [==============================] - 0s 76ms/step - loss: 0.6975 - acc: 0.6875
Epoch 4/2000
2/2 [==============================] - 0s 75ms/step - loss: 0.6606 - acc: 0.7083
Epoch 5/2000
2/2 [==============================] - 0s 83ms/step - loss: 0.6663 - acc: 0.6875
Epoch 6/2000
2/2 [==============================] - 0s 80ms/step - loss: 0.6219 - acc: 0.7292
Epoch 7/2000
2/2 [==============================] - 0s 77ms/step - loss: 0.5668 - acc: 0.7292
Epoch 8/2000
2/2 [==============================] - 0s 82ms/step - loss: 0.7564 - acc: 0.6250
Epoch 9/2000
2/2 [==============================] - 0s 79ms/step - loss: 0.9462 - acc: 0.5208
Epoch 10/2000
2/2 [==============================] - 0s 80ms/step - loss: 0.6036 - acc: 0.7500
Epoch 11/2000
2/2 [==============================] - 0s 77m

In [138]:
res = model.predict(X_test)

In [139]:
actions[np.argmax(res[0])]

'Walking'

In [140]:
actions[np.argmax(y_test[0])]

'Walking'

**8) Evaluation using Confusion Matrix and Accuracy**

In [141]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [142]:
yhat = model.predict(X_test)

In [143]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [144]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 4],
        [1, 4]],

       [[4, 1],
        [4, 3]]], dtype=int64)

In [145]:
accuracy_score(ytrue, yhat)

0.5833333333333334